<img src='../images/gdd-logo.png' width='300px' align='right' style="padding: 15px">

# <font color='#1EB0E0'>Class Based Context Managers</font>

## Introduction

In any programming language, the usage of resources like file operations or database connections is very common. But these resources are limited in supply. Therefore, the main problem lies in making sure to release these resources after usage. If they are not released then it will lead to resource leakage and may cause the system to either slow down or crash. It would be very helpful if user have a mechanism for the automatic setup and teardown of resources.In Python, it can be achieved by the usage of context managers which facilitate the proper handling of resources. The most common way of performing file operations is by using the with keyword as shown below:

## Creating a Context Manager

When creating context managers using classes, we need to ensure that the class has these methods: 
- `__enter__()`
- `__exit__()`

The `__enter__()` returns the resource that needs to be managed and the `__exit__()` does not return anything but performs the cleanup operations.

First, lets create a simple class called ContextManager to understand the basic structure of creating context managers using classes, as shown below:

In [ ]:
class ContextManager():
    def __init__(self):
        print('init method called')
          
    def __enter__(self):
        print('enter method called')
        return self
      
    def __exit__(self, type, value, traceback):
        print('exit method called')
  
  
with ContextManager() as manager:
    print('with statement block')

Our `__exit__` method accepts three arguments. They are required by every `__exit__` method which is a part of a Context Manager Class to [handle exceptions](https://book.pythontips.com/en/latest/context_managers.html#handling-exceptions) when closing files.

## File management using context manager :

Let’s apply the above concept to create a class that helps in file resource management.The FileManager class helps in opening a file, writing/reading contents and then closing it.

In [ ]:
class FileManager():
    def __init__(self, filename, mode):
        self.filename = filename
        self.mode = mode
        self.file = None
          
    def __enter__(self):
        self.file = open(self.filename, self.mode)
        return self.file
      
    def __exit__(self, exc_type, exc_value, exc_traceback):
        self.file.close()
  

In [ ]:
# loading a file 
with FileManager('test.txt', 'w') as f:
    f.write('Test')
    
print(f.closed)

On executing the with block, the following operations happen in sequence:

- A FileManager object is created with test.txt as the filename and w (write) as the mode when `__init__` method is executed.

- The `__enter__` method opens the test.txt file in write mode (setup operation) and returns the FileManager object to variable f.

- The text ‘Test’ is written into the file.

- The `__exit__` method takes care of closing the file on exiting the with block (teardown operation).
When print(f.closed) is run, the output is True as the FileManager has already taken care of closing the file which otherwise needed to be explicitly done.

## Database connection management using context manager :

Let’s create a simple database connection management system. The number of database connections that can be opened at a time is also limited(just like file descriptors). Therefore context managers are helpful in managing connections to the database as there could be chances that the programmer may forget to close the connection.

In [ ]:
from pymongo import MongoClient
  
class MongoDBConnectionManager():
    def __init__(self, hostname, port):
        self.hostname = hostname
        self.port = port
        self.connection = None
  
    def __enter__(self):
        self.connection = MongoClient(self.hostname, self.port)
        return self
  
    def __exit__(self, exc_type, exc_value, exc_traceback):
        self.connection.close()
  

In [ ]:
# connecting with a localhost
with MongoDBConnectionManager('localhost', 27017) as mongo:
    collection = mongo.connection.SampleDb.test
    data = collection.find({'_id': 1})
    print(data)
 

On executing the with block, the following operations happen in sequence:

- A MongoDBConnectionManager object is created with `'localhost'` as the hostnamename and `27017` as the port when `__init__` method is executed.
- The `__enter__` method opens the mongodb connection and returns the MongoDBConnectionManager object to variable mongo.
- The test collection in SampleDb database is accessed and the document with `_id=1` is retrieved. The name field of the document is printed.
- The `__exit__` method takes care of closing the connection on exiting the with block(teardown operation).